In [1]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import time
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans

In [2]:
판례목록_df = pd.read_html("법령검색목록.html")
판례목록_df = 판례목록_df[0]
# # 판례목록_df = 판례목록_df[(판례목록_df["판결유형"] == "민사") | (판례목록_df["판결유형"] == "일반행정") | (판례목록_df["판결유형"] == "가사") | (판례목록_df["판결유형"] == "세무")]
판례목록_df = 판례목록_df[(판례목록_df["판결유형"] == "민사")]
# for i in 판례목록_df["판례ID"]:
#     print("https://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID={}&type=HTML".format(i))
set(판례목록_df["제목"])
    

{nan,
 '상호 말소 등기절차 이행',
 '회원보증금반환등',
 '수익금교부',
 '물품대금[추완항소 기간을 도과하였는지 여부가 문제된 사건]',
 '임시집회 무효확인',
 '보증채무금·사해행위취소',
 '신용장대금청구의소',
 '소유권말소등기·위약금',
 '건물대금',
 '매매대금반환 및 손해배상',
 '가등기에기한본등기',
 '재단법인설립무효확인등',
 '토지소유권이전등기,건물철거',
 '원인무효로인한경정등기말소등기',
 '특허권침해금지 및 손해배상(기)',
 '채무부존재확인·미수금청구',
 '업무집행정지및업무대행자선임가처분',
 '결의부존재확인',
 '지입료등',
 '채권자취소등',
 '신주인수권부사채발행무효확인',
 '공사대금',
 '회사채원리금청구',
 '소유권지분 이전 등기',
 '부동산 매각 불허가 결정에 대한 이의',
 '기타소득원천징수금등',
 '매각허가취소',
 '통행방해금지등 가처분',
 '정관무효확인',
 '주지후보선출자지위확인등',
 '침해금지등',
 '소유권보존등기등말소',
 '배당이의·부당이득금 반환',
 '주주권확인',
 '주주총회결의부존재확인',
 '공사중지가처분결정',
 '위약약정금',
 '약속어음반환등',
 '소유권이전청구권가등기말소등',
 '수익증권환매대금',
 '소유권 보존 등기말소 등',
 '단기매매차익',
 '상표사용금지',
 '임차권확인등',
 '설계대금',
 '영업금지가처분',
 '건물명도·소유권이전등기',
 '임금,부당이득금',
 '예금보험금',
 '파산배당금교부청구권',
 '보수 지급등·부동산 인도등',
 '근저당권설정등기말소',
 '토지보상금,소유권이전등기',
 '분양대금등반환',
 '예금반환등',
 '소송이송신청기각',
 '가압류취소(가압류취소에대한즉시항고)',
 '채권조사확정재판에대한이의의소',
 '주주대표소송',
 '영업금및 보증금 반환',
 '주지임명무효등확인',
 '임대료반환등',
 '정리채권확정의소',
 '명의개서신청절차이행',
 '자동차인도',
 '정정·반론(PD수첩 사건)',
 '가압류에서본압

In [8]:
판례목록_하등_df = pd.read_html("법령검색목록(1).html")
판례목록_하등_df = 판례목록_하등_df[0]
name_list = [str(i) for i in 판례목록_하등_df["제목"] if "손해배상" in str(i)]
name_list = list(set(name_list))
손해배상_하등_df = 판례목록_하등_df[판례목록_하등_df["제목"] == name_list[0]]
손해배상_하등_df
for idx,i in enumerate(name_list):
    if idx==0:
        continue
    손해배상_하등_df = 손해배상_하등_df.append((판례목록_하등_df[판례목록_하등_df["제목"] == i]),ignore_index=True)
손해배상_하등_df

,순번,판례ID,제목,법원,사건유형,판결유형,선고일자
0,5176,165171,손해배상(기)등·손해배상(기),부산가법,하급심,민사,2012.05.31
1,68,205564,채무부존재확인·손해배상(기),광주고등법원,하급심,민사,2015.11.19
2,7404,188432,채무부존재확인·손해배상(기),서울고등법원,하급심,민사,2016.09.22
3,16782,183167,채무부존재확인·손해배상(기),서울중앙지방법원,하급심,민사,2013.11.19
4,22326,205510,채무부존재확인·손해배상(기),전주지방법원 남원지원,하급심,민사,2015.01.22
...,...,...,...,...,...,...,...
2895,15666,166435,손해배상 등,서울동부지법,하급심,민사,2007.05.21
2896,17785,163584,손해배상 등,서울중앙지법,하급심,민사,2012.03.29
2897,7472,217121,손해배상(기)등ㆍ손해배상(기),서울고등법원,하급심,민사,2016.06.17
2898,17697,214521,손해배상청구등의소,서울중앙지법,하급심,민사,2020.08.27


In [ ]:
판례목록_하등_df
# 판례_하등 XML 상세조회
info_하등 = []
def crawling_func(url):
    global info
    text_판시사항 = ""
    text_판결요지 = ""
    text_참조조문 = ""
    text_참조판례 = ""
    text_주문 = ""
    text_청구취지 = ""
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)
        판시사항_soup = BeautifulSoup(soup.판시사항.get_text(),'lxml')
#         info.append(판시사항_soup.get_text())
        text_판시사항+=판시사항_soup.get_text()
        판결요지_soup = BeautifulSoup(soup.판결요지.get_text(),'lxml')
#         info.append(판결요지_soup.get_text())
        text_판결요지+=판결요지_soup.get_text()
        참조조문_soup = BeautifulSoup(soup.참조조문.get_text(),'lxml')
        for i in 참조조문_soup:
            try:
                text_참조조문+=i.get_text()
            except:
                pass
        참조판례_soup = BeautifulSoup(soup.참조판례.get_text(),'lxml')
        text_참조판례+=참조판례_soup.get_text()
        
        # 주문 crawling
        response = requests.get(url2)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml-xml')
        try:
            i_list = soup.find("div",attrs={"class":"pgroup","id":"conScroll"}).find_all("p",attrs={"class":"pty4_dep1"})
            for i in i_list:
                if i.next_sibling.next_sibling.get_text()[1:-1]=="주 문":
                    text_주문+=i.next_sibling.next_sibling.next_sibling.next_sibling.get_text()
                    break
        except:
            pass
        try:
            i_list = soup.find("div",attrs={"class":"pgroup","id":"conScroll"}).find_all("p",attrs={"class":"pty4_dep1"})
            for i in i_list:
                if i.next_sibling.next_sibling.get_text()[1:-1]=="청구취지 및 항소취지" or i.next_sibling.next_sibling.get_text()[1:-1]=="청구취지":
                    text_청구취지 += i.next_sibling.next_sibling.next_sibling.next_sibling.get_text()
                    break
        except:
            pass
        info_하등.append([ID,text_판시사항,text_판결요지,text_참조조문,text_참조판례,text_주문,text_청구취지])
#         print(info_하등)
    except:
        print(url,"오류발생, 재귀호출 시작")
        time.sleep(1)
        crawling_func(url)
    finally:
        return

for idx,ID in enumerate(손해배상_하등_df["판례ID"]):   
    #기본 url
    url = "http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID={}&type=XML".format(ID)
    url2 = "https://www.law.go.kr/precInfoP.do?mode=0&precSeq={}&vSct=*".format(ID)
    crawling_func(url)
    print(idx,ID)

# info_하등


0 165171
1 205564
2 188432
3 183167
4 205510
5 167374
6 158528
7 74874
8 74855
9 75007
10 71848
11 71846
12 78213
13 120823
14 158558
15 120176
16 133481
17 173372
18 173340
19 71164
20 192385
21 118780
22 172804
23 70572
24 126030
25 172135
26 172145
27 126026
28 118303
29 69820
30 119193
31 118253
32 118538
33 119274
34 119294
35 119276
36 118460
37 184658
38 205496
39 65017
40 172253
41 118611
42 118444
43 152088
44 152186
45 78445
46 172872
47 172860
48 118853
49 118686
50 217159
51 217051
52 69915
53 71871
54 206171
55 212425
56 208477
57 159788
58 138226
59 207164
60 176127
61 176233
62 76752
63 140320
64 176231
65 142947
66 69740
67 69676
68 205652
69 177819
70 72156
71 146042
72 77021
73 76919
74 76837
75 76831
76 76760
77 76756
78 146045
79 77025
80 77024
81 76834
82 76833
83 76827
84 76826
85 76822
86 146013
87 76879
88 76859
89 76778
90 76678
91 118851
92 146034
93 76954
94 140689
95 182695
96 198557
97 198552
98 73539
99 75804
100 120531
101 163296
102 163284
103 214537
104

822 71946
823 71944
824 71945
825 71942
826 71932
827 71906
828 71901
829 71898
830 71868
831 71805
832 119039
833 76882
834 76874
835 76873
836 76769
837 76251
838 76170
839 76147
840 76154
841 75974
842 75956
843 75747
844 75743
845 75738
846 75729
847 75726
848 75668
849 75665
850 75659
851 75658
852 75656
853 75653
854 75651
855 75646
856 75647
857 75643
858 76941
859 76881
860 76336
861 76259
862 76257
863 76166
864 75762
865 75633
866 75623
867 76338
868 76169
869 118694
870 76847
871 76782
872 76672
873 76604
874 76600
875 76431
876 76340
877 76254
878 76253
879 76249
880 76158
881 76072
882 76070
883 75862
884 75648
885 75649
886 75631
887 75625
888 75961
889 75770
890 75735
891 76867
892 76763
893 75976
894 75858
895 76157
896 76964
897 76950
898 74515
899 73962
900 76222
901 75206
902 75188
903 74963
904 74463
905 74070
906 73487
907 72945
908 75748
909 76820
910 76643
911 75929
912 75532
913 190181
914 75356
915 75178
916 75144
917 75136
918 75123
919 74849
920 74980
921 771

In [27]:
info_하등법령_df = pd.DataFrame(info_하등,columns = ["ID","판시사항","판결요지","참조조문","참조판례","주문","청구취지"])
# info_하등법령_df["판결유형"] = 판례목록_하등_df["ID"]
info_하등법령_df

,ID,판시사항,판결요지,참조조문,참조판례,주문,청구취지
0,165171,甲이 배우자 乙의 성기능 장애로 인하여 사실혼관계가 단기간에 파탄되었다는 이유로 乙...,甲이 배우자 乙의 성기능 장애로 인하여 사실혼관계가 단기간에 파탄되었다는 이유로 乙...,민법 제806조,,"1. 피고(반소원고) 1은 원고(반소피고)에게, 가. 20,000,000원과 ...","1. 본소 가. 주위적 청구취지피고(반소원고, 이하 ‘피고’라 한다)들은 연대하여..."
1,205564,,,,,1. 제1심 판결을 다음과 같이 변경한다. 가. 2011. 6. 2. 12:3...,1. 청구취지 가. 본소 2011. 6. 2. 12:30경 남원시 (주소 생략)...
